# &AElig;co
***Calc module***
<br/><br/>

✈️♻️

This code is part of the Master's Thesis "A Life Cycle Approach for Aircraft Ecodesign", by Giácomo Parolin.

``Please enter the following information:``

In [1]:
import os
os.chdir("C:/Users/giparoli/Documents/Projetos/AEco")

import numpy as np

aircraft_type = 'pax'  #pax or cargo

input_path = './Data/A320_inputs.xlsx'
input_sheet = 'A320'
# output_path = './Outputs/'+ input_sheet + '_outputs'
database_path = './Data/database_A320.xlsx'

iterations = 1000
chunks = 1000  #'auto' or more than 500

In [2]:
from Tools import *
from Model import *

In [3]:
UP_dataframe = read_unit_processes(database_path) #reads Unit Processes from database
UP = unit_process_dataset(UP_dataframe) #creates Unit Processes xarray dataset

In [4]:
MP_data, EP_data = read_CF(database_path) #reads MP and EP factors from database
CF = CharactFactors(MP_data, EP_data, UP.Substances) #creates CharactFactors object
CF.MP.dataset['NLT'].data = np.negative(CF.MP.dataset['NLT'].data) #adjusting the NLT category CFs.
CF.MP.to_array();

# Analysis

In [29]:
inputs = read_inputs(input_path, input_sheet) #reads inputs
p = ParameterSet(inputs, iterations, chunks) #creates ParameterSet object
p = func_unit(aircraft_type, p) #creates funcional unit parameters (see Tools.py)

In [30]:
baseline = 0.11047247
weight2ff = 2 # x de redução em ff ocorre através da weight2ff*x redução em weight
ff_reduct = np.linspace(0,0.10,9)
weight_reduct = ff * weight2ff

ff_ccd = p["ff_ccd"]
OEW = p["OEW"]

for i in range(99):
    ff = ff_ccd * (1 - ff_reduct[i])
    weight = OEW * (1-weight_reduct[i])
    
    # p["ff_ccd"] = ff
    p["OEW"] = weight

    inventory = LCI(name=input_sheet, type=aircraft_type, iterations=iterations, UP=UP, parameters=p) #creates empty LCI object
    inventory.run(); #runs LCI model (see Model.py)
    aeco = LCIA.build(inventory, CF) #creating LCIA object
    
    CC_result = aeco.mean(by="sum").loc["CC"].compute().data

    if CC_result > baseline:
        print(i)
        continue
    else:
        print("Chegou no resultado!")
        CC_offset = CC_result
        weight_offset = weight_reduct[i]
        ff_offset = ff_reduct[i]
        i_offset = i
        break

print(f"o resultado de CC é {CC_offset}")
print(f"o offset de peso necessário é {weight_offset*100}%, com offset de consumo de combustível de {ff_offset*100}%")
print(f"isso foi obtido na {i}-ésima iteração")

0
1
2
3
4
5
Chegou no resultado!
o resultado de CC é 0.11045429288752395
o offset de peso necessário é 15.000000000000002%, com offset de consumo de combustível de 7.500000000000001%
isso foi obtido na 6-ésima iteração


In [23]:
ff_reduct[1]*100

0.05102040816326531

In [62]:
(CC_result - baseline) / baseline * 100 #change from baseline in %

-8.141148510390261e-07

# Export Results

In [53]:
aeco.save(output_path, LCI=False) #Export MP, EP, LCI and CTV result to NetCDF file.

In [22]:
#aeco.to_excel(output_path)